# How to run a prompt plugins from file

Now that you're familiar with Kernel basics, let's see how the kernel allows you to run Prompt Plugins and Prompt Functions stored on disk.

A Prompt Plugin is a collection of Semantic Functions, where each function is defined with natural language that can be provided with a text file.

Refer to our [glossary](https://github.com/microsoft/semantic-kernel/blob/main/docs/GLOSSARY.md) for an in-depth guide to the terms.

The repository includes some examples under the [samples](https://github.com/microsoft/semantic-kernel/tree/main/samples) folder.

For instance, [this](../../../prompt_template_samples/FunPlugin/Joke/skprompt.txt) is the **Joke function** part of the **FunPlugin plugin**:


```
WRITE EXACTLY ONE JOKE or HUMOROUS STORY ABOUT THE TOPIC BELOW.
JOKE MUST BE:
- G RATED
- WORKPLACE/FAMILY SAFE
NO SEXISM, RACISM OR OTHER BIAS/BIGOTRY.
BE CREATIVE AND FUNNY. I WANT TO LAUGH.
+++++
{{$input}}
+++++
```


Note the special **`{{$input}}`** token, which is a variable that is automatically passed when invoking the function, commonly referred to as a "function parameter".

We'll explore later how functions can accept multiple variables, as well as invoke other functions.


In the same folder you'll notice a second [config.json](../../../prompt_template_samples/FunPlugin/Joke/config.json) file. The file is optional, and is used to set some parameters for large language models like Temperature, TopP, Stop Sequences, etc.

```
{
  "schema": 1,
  "description": "Generate a funny joke",
  "execution_settings": {
    "default": {
      "max_tokens": 1000,
      "temperature": 0.9,
      "top_p": 0.0,
      "presence_penalty": 0.0,
      "frequency_penalty": 0.0
    }
  },
  "input_variables": [
    {
      "name": "input",
      "description": "Joke subject",
      "default": ""
    },
    {
      "name": "style",
      "description": "Give a hint about the desired joke style",
      "default": ""
    }
  ]
}

```


Given a prompt function defined by these files, this is how to load and use a file based prompt function.

Load and configure the kernel, as usual, loading also the AI service settings defined in the [Setup notebook](00-getting-started.ipynb):


In [ ]:
!python -m pip install semantic-kernel==1.0.5

In [3]:
import sys
import os

# Add the samples directory to the Python path
samples_dir = r'C:\Users\sulma\OneDrive\Documents\GitHub\semantic-kernel\python\samples'
sys.path.append(samples_dir)

try:
    from services import Service
    from service_settings import ServiceSettings

    service_settings = ServiceSettings(
        global_llm_service="AzureOpenAI")  # Example setting

    # Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
    selectedService = (
        Service.AzureOpenAI
        if service_settings.global_llm_service is None
        else Service(service_settings.global_llm_service.lower())
    )
    print(f"Using service type: {selectedService}")
except ModuleNotFoundError as e:
    print(f"Error: {e}")
    print("Ensure that 'service_settings.py' is in the correct directory and properly named.")
    # Additional debugging information
    print("Current Python path:")
    for path in sys.path:
        print(path)

Using service type: Service.AzureOpenAI


In [5]:
from dotenv import load_dotenv
from semantic_kernel import Kernel

kernel = Kernel()


env_path = r'C:\Users\sulma\OneDrive\Documents\GitHub\semantic-kernel\python\samples\getting_started\.env'
load_dotenv(env_path)

service_id = None

# Load the API keys and other settings from the environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_org_id = os.getenv('OPENAI_ORG_ID')
azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
azure_chat_deployment_name = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')



if selectedService == Service.OpenAI:

    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"

    kernel.add_service(


        OpenAIChatCompletion(service_id=service_id,
                             ai_model_id="gpt-3.5-turbo-1106"),


    )


elif selectedService == Service.AzureOpenAI:

    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"

    kernel.add_service(


        AzureChatCompletion(
            service_id=service_id,
            api_key=azure_openai_api_key,           # Use the api_key from the .env file
            # Use the deployment_name from the .env file
            deployment_name=azure_chat_deployment_name,
            endpoint=azure_openai_endpoint,         # Use the endpoint from the .env file
            env_file_path=env_path,




        )
    )

Import the plugin and all its functions:


In [6]:
# note: using plugins from the samples folder
plugins_directory = "../../../prompt_template_samples/"

funFunctions = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="FunPlugin")

jokeFunction = funFunctions["Joke"]

How to use the plugin functions, e.g. generate a joke about "_time travel to dinosaur age_":


In [7]:
result = await kernel.invoke(jokeFunction, input="travel to dinosaur age", style="silly")
print(result)

Why did the T-Rex go on vacation to the future? 
Because he heard the steaks were even bigger!


Great, now that you know how to load a plugin from disk, let's show how you can [create and run a prompt function inline.](./03-prompt-function-inline.ipynb)
